## Agenda
- 합, 누적합
- 상관관계, 공분산
- 중복값 처리

## 기본 설정

In [ ]:
import pandas as pd
import numpy as np

## 1.3 기초 통계 함수

* `sum()`
* `mean()`
* `std()`
* `idxmax()`/`idxmin()`
* `cumsum()`
* `describe()`

- 기본적으로 열 단위로 작동하며
- 결측치는 행 또는 열의 모든 값이 결측치가 아니라면 기본적으로 무시되어 계산되지 않는다.
- 행 단위로 작동하게 하려면 축을 `axis=1` 또는 `axis='columns`로 지정
- 결측치를 무시하지 않으려면 `skipna=False`로 지정(무시하면 계산 결과가 나타나지 않는다.)

In [ ]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])

In [ ]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


* `sum()` 메서드: 행/열 단위 합 계산

In [ ]:
df.sum()

one    9.25
two   -5.80
dtype: float64

In [ ]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [ ]:
df.sum(axis="columns")

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [ ]:
# 결측치를 무시하지 않으면 계산 되지 않는다. 
df.sum(skipna=False)

one   NaN
two   NaN
dtype: float64

In [ ]:
#Series는 하나의 열을 갖는 DataFrame과 같은 방식으로 동작한다. 
df["one"].sum()

9.25

- `mean()` 메서드: 평균값 계산
  - 전체를 더해서 전체의 길이만큼 나눠준다. 
  - 수학 : 20, 국어 : 10, 영어: 30 
  - 전체합 : 60, 평균 : 60/3 = 20

In [ ]:
df.mean()

one    3.083333
two   -2.900000
dtype: float64

In [ ]:
df.mean(axis = 1)

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

In [ ]:
df.mean(axis = "columns")

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

In [ ]:
df.mean(skipna=False)

one   NaN
two   NaN
dtype: float64

In [ ]:
df["two"].mean(skipna=False)

nan

In [ ]:
df["two"].mean()

-2.9

- `std()` 메서드: 표준편차 계산
$$\sigma= \sqrt{ \frac{{(x_1 -m)}^{2} + {(x_2 -m)}^{2} + \cdot \cdot \cdot + {(x_N -m)}^{2} }{N} } $$
- `var` 분산 : 평균으로 부터의 거리의 제곱에 대한 평균
$$ V= \frac{{(x_1 -m)}^{2} + {(x_2 -m)}^{2} + \cdot \cdot \cdot + {(x_N -m)}{2} }{N} $$

In [ ]:
df.std()

one    3.493685
two    2.262742
dtype: float64

In [ ]:
df.std(axis = 1)

a         NaN
b    8.202439
c         NaN
d    1.449569
dtype: float64

* `idxmax()`/`idxmin()`: 최댓값/최솟값을 갖는 인덱스 확인

In [ ]:
df.idxmax()

one    b
two    d
dtype: object

In [ ]:
df.idxmin()

one    d
two    b
dtype: object

* `cumsum()`: 누적 합 계산

In [ ]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [ ]:
df.cumsum(skipna=False)

,one,two
a,1.4,NaN
b,8.5,NaN
c,NaN,NaN
d,NaN,NaN


* `describe()`: 요약 통계 보여주기

수치형 데이터의 경우 평균값, 표준편차, 사분위수 등의 통계 정보를 요약해서 보여준다.

In [ ]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


수치형 데이터가 아닐 경우 다른 요약 통계를 보여준다.

In [ ]:
ser = pd.Series(['a', 'a', 'b', 'c'] * 2)

In [ ]:
ser.describe()

count     8
unique    3
top       a
freq      4
dtype: object

### 1.3.1 상관관계와 공분산

4 개 회사의 주가(price)와 거래량(volume)을 담고 있는 두 개의 데이터를 이용하여 상관계수와 공분산을 계산해보자.

- pkl 파일: 판다스에서 제공하는 객체를 `to_pickle()` 메서드를 이용하여 
    컴퓨터에 파일로 저장할 때 사용되는 바이너리 파일.
- `read_pickle()`: 저장된 pkl 파일을 파이썬으로 불러오는 함수

아래 코드는 일별 주가 데이터를 불러온다.
2010년 1월 4일부터 2016년 10월 21일까지의 데이터 1714개를 담고 있다.

In [ ]:
price = pd.read_pickle('./5_yahoo_price.pkl')
price

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
...,...,...,...,...
2016-10-17,117.550003,779.960022,154.770004,57.220001
2016-10-18,117.470001,795.260010,150.720001,57.660000
2016-10-19,117.120003,801.500000,151.259995,57.529999


아래 코드는 동일 회사, 동일 날짜의 1일 거래량(volume) 담고 있는 데이터를 불러온다.

In [ ]:
volume = pd.read_pickle('./5_yahoo_volume.pkl')
volume

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,123432400,3927000,6155300,38409100
2010-01-05,150476200,6031900,6841400,49749600
2010-01-06,138040000,7987100,5605300,58182400
2010-01-07,119282800,12876600,5840600,50559700
2010-01-08,111902700,9483900,4197200,51197400
...,...,...,...,...
2016-10-17,23624900,1089500,5890400,23830000
2016-10-18,24553500,1995600,12770600,19149500
2016-10-19,20034600,116600,4632900,22878400


주가의 일단위 변화율을 알아보기 위해 퍼센트 변화율을 확인해보자.

__참고:__ 증권분야에서 return은 이익율을 의미한다.

In [ ]:
returns = price.pct_change()
returns.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-17,-0.000680,0.001837,0.002072,-0.003483
2016-10-18,-0.000681,0.019616,-0.026168,0.007690
2016-10-19,-0.002979,0.007846,0.003583,-0.002255
2016-10-20,-0.000512,-0.005652,0.001719,-0.004867
2016-10-21,-0.003930,0.003011,-0.012474,0.042096


####  `corr()`/`cov()` 메서드

상관계수와 공분산 모두 두 확률변수 사이의 선형관계를 보여주며
차이점은 다음과 같다.

- 공분산: 두 확률변수 $X, Y$ 사이의 선형관계
    - 양수/음수 여부에 따라 양 또는 음의 선형관계
    - 절댓값이 클 수록 강한 선형관계를 갖고있다. 
    - 다만, 사용되는 확률변수의 척도(scale)에 많은 영향을 많이받아 정규화된 값인 상관계수를 주로 사용한다. 

$$
\begin{align*}
Cov(X, Y) & = E((X-\mu_X)(Y-\mu_Y))\\[2ex]
\mu_X & = E(X) = \dfrac{\sum X}{n}\\[1.5ex]
\mu_Y & = E(Y) = \dfrac{\sum Y}{n}
\end{align*}
$$

- 상관계수: 두 확률변수 사이의 선형관계를 -1과 1 사이의 값으로 표현.
    - 양수/음수 여부에 따라 양 또는 음의 선형관계
    - 절댓값이 1에 가까울 수록 강한 선형관계
$$
\begin{align*}
\rho & = \frac{Cov(X, Y)}{\sigma_X\cdot \sigma_Y}\\[2ex]
\sigma_X & = \sqrt{Var(X)}\\[1.5ex]
\sigma_X & = \sqrt{Var(X)}\\[1.5ex]
Var(X) & = \dfrac{\sum (X-\mu_X)^2}{n}\\[1.5ex]
Var(Y) & = \dfrac{\sum (X-\mu_Y)^2}{n}
\end{align*}
$$    

'MSFT'와 'IBM' 사이의 상관계수

In [ ]:
returns["MSFT"].corr(returns["IBM"])

0.49976361144151144

'MSFT'와 'IBM' 사이의 공분산

In [ ]:
returns["MSFT"].cov(returns["IBM"])

8.870655479703546e-05

모든 회사들 간의 상관계수와 공분산

In [ ]:
returns.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.000277,0.000107,0.000078,0.000095
GOOG,0.000107,0.000251,0.000078,0.000108
IBM,0.000078,0.000078,0.000146,0.000089
MSFT,0.000095,0.000108,0.000089,0.000215


In [ ]:
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.407919,0.386817,0.389695
GOOG,0.407919,1.000000,0.405099,0.465919
IBM,0.386817,0.405099,1.000000,0.499764
MSFT,0.389695,0.465919,0.499764,1.000000


#### `corrwith()`  
- 다른 시리즈 또는 데이터프레임과의 상관계수 계산
- 시리즈를 인자로 사용하면 각 열에 대한 상관계수를 계산

In [ ]:
# IBM과 returns DF와의 상관계수를 계산한다. 
returns.corrwith(returns.IBM)

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

- 데이터프레임에 대해서는 공통 인덱스를 사용하는 모든 열에 대한 상관계수를 계산

In [ ]:
returns.corrwith(volume)

AAPL   -0.075565
GOOG   -0.007067
IBM    -0.204849
MSFT   -0.092950
dtype: float64

### 1.3.2 중복과 빈도수

#### `unique()`

- Series에서 사용된 값을 중복 없이 확인하려면 `unique()` 메서드 사용 
- `set()` 함수와 유사하게 동작, 넘파이 어레이를 반환

In [ ]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [ ]:
obj.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

#### `value_counts()`
- 값들의 빈도수를 확인하기 위해 사용

In [ ]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64